# [모듈 3.1] Code Pipeline 생성 하기

## 참고 자료
- https://docs.aws.amazon.com/codepipeline/latest/userguide/pipelines-create.html


# 1. 환경 설정


In [232]:
%store -r repository_name
%store -r code_build_project_name
%store -r account_id
%store -r region
%store -r bucket



# 2. 파라미터 설정

In [233]:
code_pipeline_name = "ncf-training-code-pipeline"
# pipeline_role_name = 'arn:aws:iam::189546603447:role/service-role/AWSCodePipelineServiceRole-us-east-1-hello-iris-clone'
# pipeline_role_name = "arn:aws:iam::057716757052:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"
pipeline_role_name = "arn:aws:iam::057716757052:role/code-pipeline-gsmoon"
branch_name = 'master'
# code_build_project_name = "iris-build-project"
# S3ObjectKey = "ExampleCodePipelineSampleBundle.zip"


# 3. 파이프라인 JSON 정의

In [234]:
pipeline_dic = {
    "pipeline": {
        "name": code_pipeline_name,
        "roleArn": pipeline_role_name,
        "artifactStore": {
            "type": "S3",
            "location": bucket
        },
        "stages": [
            {
                "name": "Source",
                "actions": [
                    {
                        "name": "Source",
                        "actionTypeId": {
                            "category": "Source",
                            "owner": "AWS",
                            "provider": "CodeCommit",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "BranchName": branch_name,
                            "OutputArtifactFormat": "CODE_ZIP",
                            "PollForSourceChanges": "false",
                            "RepositoryName": repository_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "inputArtifacts": [],
                        "region": region,
                        "namespace": "SourceVariables"
                    }
                ]
            },
            {
                "name": "Build",
                "actions": [
                    {
                        "name": "BuildAndExecuteSageMakerPipeline",
                        "actionTypeId": {
                            "category": "Build",
                            "owner": "AWS",
                            "provider": "CodeBuild",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "ProjectName": code_build_project_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "BuildArtifact"
                            }
                        ],
                        "inputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "region": region,
                        "namespace": "BuildVariables"
                    }
                ]
            }
        ],
        "version": 1
    },
}

## 3.1. Dic 포맷을 JSON 으로 변경

In [235]:
import json

pipeline_json = json.dumps(pipeline_dic)
json.loads(pipeline_json)

{'pipeline': {'name': 'ncf-training-code-pipeline',
  'roleArn': 'arn:aws:iam::057716757052:role/code-pipeline-gsmoon',
  'artifactStore': {'type': 'S3',
   'location': 'sagemaker-us-east-1-057716757052'},
  'stages': [{'name': 'Source',
    'actions': [{'name': 'Source',
      'actionTypeId': {'category': 'Source',
       'owner': 'AWS',
       'provider': 'CodeCommit',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'BranchName': 'master',
       'OutputArtifactFormat': 'CODE_ZIP',
       'PollForSourceChanges': 'false',
       'RepositoryName': 'mlops-ncf'},
      'outputArtifacts': [{'name': 'SourceArtifact'}],
      'inputArtifacts': [],
      'region': 'us-east-1',
      'namespace': 'SourceVariables'}]},
   {'name': 'Build',
    'actions': [{'name': 'BuildAndExecuteSageMakerPipeline',
      'actionTypeId': {'category': 'Build',
       'owner': 'AWS',
       'provider': 'CodeBuild',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'Project

## 3.2. JSON 파일 저장

In [236]:
json_file_path = 'src/hello_pipeline.json'
with open(json_file_path, "w") as outfile:
    outfile.write(pipeline_json)

In [237]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

{'pipeline': {'name': 'ncf-training-code-pipeline',
  'roleArn': 'arn:aws:iam::057716757052:role/code-pipeline-gsmoon',
  'artifactStore': {'type': 'S3',
   'location': 'sagemaker-us-east-1-057716757052'},
  'stages': [{'name': 'Source',
    'actions': [{'name': 'Source',
      'actionTypeId': {'category': 'Source',
       'owner': 'AWS',
       'provider': 'CodeCommit',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'BranchName': 'master',
       'OutputArtifactFormat': 'CODE_ZIP',
       'PollForSourceChanges': 'false',
       'RepositoryName': 'mlops-ncf'},
      'outputArtifacts': [{'name': 'SourceArtifact'}],
      'inputArtifacts': [],
      'region': 'us-east-1',
      'namespace': 'SourceVariables'}]},
   {'name': 'Build',
    'actions': [{'name': 'BuildAndExecuteSageMakerPipeline',
      'actionTypeId': {'category': 'Build',
       'owner': 'AWS',
       'provider': 'CodeBuild',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'Project

# 4. 파이프라인 실행

In [238]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
echo $PWD
aws codepipeline create-pipeline --cli-input-json file://$json_file_path
# aws codepipeline create-pipeline --cli-input-json file://src/hello_pipeline.json

src/hello_pipeline.json
/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/3_sm-train-codepipeline
{
    "pipeline": {
        "name": "ncf-training-code-pipeline",
        "roleArn": "arn:aws:iam::057716757052:role/code-pipeline-gsmoon",
        "artifactStore": {
            "type": "S3",
            "location": "sagemaker-us-east-1-057716757052"
        },
        "stages": [
            {
                "name": "Source",
                "actions": [
                    {
                        "name": "Source",
                        "actionTypeId": {
                            "category": "Source",
                            "owner": "AWS",
                            "provider": "CodeCommit",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "BranchName": "master",
                            "OutputArtifactFormat": "CODE

# 5. 코드 파이프라인 실행 상태 확인

In [239]:
%%sh -s {code_pipeline_name}
code_pipeline_name=$1
echo $code_pipeline_name
aws codepipeline get-pipeline-state --name $code_pipeline_name

ncf-training-code-pipeline
{
    "pipelineName": "ncf-training-code-pipeline",
    "pipelineVersion": 1,
    "stageStates": [
        {
            "stageName": "Source",
            "inboundTransitionState": {
                "enabled": true
            },
            "actionStates": [
                {
                    "actionName": "Source",
                    "latestExecution": {
                        "actionExecutionId": "3e4a9996-ceb2-4083-ae76-f3d49b099fda",
                        "status": "InProgress",
                        "lastStatusChange": 1657282929.091
                    },
                    "entityUrl": "https://console.aws.amazon.com/codecommit/home?region=us-east-1#/repository/mlops-ncf/browse/master/--/"
                }
            ],
            "latestExecution": {
                "pipelineExecutionId": "54b9e32b-64ab-4679-a868-51261f9ff64b",
                "status": "InProgress"
            }
        },
        {
            "stageName": "Build",
  

In [240]:
%store code_pipeline_name

Stored 'code_pipeline_name' (str)
